In [91]:
import math

import geopandas as gpd
import laspy
import numpy as np
import scipy as sp
import startinpy

import raster
from dev.utils import timing

In [2]:
class PointCloud:
    def __init__(self, filename: str) -> None:
        with laspy.open(filename) as f:
            self.las = f.read()

    def __len__(self) -> int:
        return len(self.las.points)

    def __getitem__(self, key: int):
        ...

In [3]:
class BufferableGeoDataFrame(gpd.GeoDataFrame):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()

    def buffer(self, *args, **kwargs) -> None:
        ...

# Method 1
1. Fetch the footprints of all buildings in the tile.
2. Buffer the footprints by 10 m.
    - The buffer distance is selected such that:
        - The resulting raster datasets (Step 6) do not contain empty cells when masked
        - Most if not all the natural neighbors of the points along the footprint edges are included in the subsequent steps.
3. Intersect the point cloud with the buffers.
4. Compute the Delaunay triangulation of the remaining points.
5. Compute and rasterize the gridded slope field corresponding to the triangulation.
6. Rasterize the reflectance field corresponding to the triangulation.

In [4]:
@timing
def read(filename: str) -> PointCloud:
    return PointCloud(filename)


# FIXME: Rewrite this function as a PointCloud method.
@timing
def to_gdf(pc: PointCloud) -> gpd.GeoDataFrame:
    return gpd.GeoDataFrame({"id": np.arange(len(pc)), "geometry": gpd.points_from_xy(pc.las.x, pc.las.y)},
                            crs="EPSG:28992")


@timing
# TODO: Review the various buffer options.
# TODO: Create a BufferableGeoDataFrame and rewrite this function as a class method?
def buffer(objs: gpd.GeoDataFrame, dist: float = 10) -> None:
    objs["geometry"] = objs["geometry"].buffer(dist)


# FIXME: Rewrite this function as a PointCloud method.
@timing
def intersect(pc: PointCloud, objs: gpd.GeoDataFrame) -> np.ndarray:
    return to_gdf(pc).overlay(objs)["id_1"].to_numpy()


# FIXME: Rewrite this function as a PointCloud method.
@timing
def triangulate(dt: startinpy.DT, pts: laspy.PackedPointRecord, sc: str = "z") -> None:
    dt.insert(np.vstack((pts.x, pts.y, pts[sc])).transpose())

In [5]:
pc = read("../test.LAZ")

func read :-> 48 ms


In [6]:
# Step 1
bx = gpd.read_file("C:/Documents/RoofSense/data/temp/9-284-556.buildings.gpkg")

# Step 2
buffer(bx)

func buffer :-> 123 ms


In [7]:
# Step 3.1
ids = intersect(pc, bx)

func to_gdf :-> 2354 ms
func intersect :-> 27183 ms


In [8]:
# Step 3.2
# FIXME: Integrate this operation into the intersection step.
# TODO: Overwrite the original point cloud?
pts = pc.las.points[ids]

In [73]:
# Step 4
dt = startinpy.DT()
triangulate(dt, pts, sc="Reflectance")

func triangulate :-> 5892 ms


array([[            inf,             inf,             inf],
       [ 8.54999740e+04,  4.46262433e+05, -1.31600000e+01],
       [ 8.54999150e+04,  4.46262202e+05, -1.33900000e+01],
       ...,
       [ 8.55581250e+04,  4.46262279e+05, -5.99000000e+00],
       [ 8.55578140e+04,  4.46262193e+05, -6.27000000e+00],
       [ 8.55575830e+04,  4.46262197e+05, -5.48000000e+00]])

In [52]:
# Step 5
# FIXME: Promote this constant to an environment variable.
GRID_SIZE = 0.25

# FIXME: Rewrite the following cells as Raster methods.
# Initialize the raster grid.
grid = raster.Raster(GRID_SIZE, dt.get_bbox())

# Gather the center coordinates of the grid cells.
cnts = []
for row in range((grid.len_y - 1), -1, -1):
    y = grid.bbox[1] + (row * GRID_SIZE) + (0.5 * GRID_SIZE)
    for col in range(grid.len_x):
        x = grid.bbox[0] + (col * GRID_SIZE) + (0.5 * GRID_SIZE)
        cnts.append([x, y])

In [53]:
# Interpolate the field values at the cell centers.
vals = dt.interpolate({"method": "Laplace"}, cnts)

In [54]:
# Populate the raster.
grid._Raster__data = vals.reshape((grid.len_y, grid.len_x))

In [55]:
# Save the raster.
grid.save("../test_reflectance.tiff")

In [79]:
# FIXME: Rewrite the following cells as PointCloud methods.
# Fetch the points to triangulate.
test = np.vstack((pts.x, pts.y, pts.Reflectance)).transpose()
# Sort the points in descending order according to their elevation.
# TODO: Use the scaled instead of the recorded elevation?
test = test[pts.Z.argsort()[::-1]]
# Find the points with the same planar projection.
_, unq_ids = np.unique(test[:, :-1], axis=0, return_index=True)
# Discard the points.
test = test[unq_ids]

test

array([[ 8.52949800e+04,  4.46294082e+05, -3.27000000e+00],
       [ 8.52949800e+04,  4.46494810e+05, -6.20000000e+00],
       [ 8.52949800e+04,  4.46512507e+05, -5.79000000e+00],
       ...,
       [ 8.56359910e+04,  4.46337425e+05, -6.67000000e+00],
       [ 8.56360040e+04,  4.46337728e+05, -5.41000000e+00],
       [ 8.56360200e+04,  4.46337156e+05, -6.70000000e+00]])

In [ ]:
for i in range(100, len(pts) + 101, 100):
    dm = sp.spatial.distance_matrix(np.vstack((pts.x, pts.y)).transpose()[i:i + 100],
                                    np.vstack((pts.x, pts.y)).transpose())
    tol = np.min(dm[np.nonzero(dm)])
    if tol < 0.001:
        print(i, tol)

3600 0.0009999999892897904
8500 0.0009999999892897904
16000 0.0009999999892897904
29400 0.0009999999892897904
37600 0.0009999999892897904


In [88]:
dt1 = startinpy.DT()
# TOSELF: Is this hack guaranteed to work or should a more reasonable value be used?
dt1.snap_tolerance = math.ulp(0)
# TOSELF: Does the insertion order affect performance?
dt1.insert(np.vstack((pts.x, pts.y, pts.Reflectance)).transpose())

dt1.points

array([[            inf,             inf,             inf],
       [ 8.54999740e+04,  4.46262433e+05, -1.31600000e+01],
       [ 8.54999150e+04,  4.46262202e+05, -1.33900000e+01],
       ...,
       [ 8.55581250e+04,  4.46262279e+05, -5.99000000e+00],
       [ 8.55578140e+04,  4.46262193e+05, -6.27000000e+00],
       [ 8.55575830e+04,  4.46262197e+05, -5.48000000e+00]])

In [89]:
print(len(test) == len(dt1.points[1:]))

True
